In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

cf = SparkConf()
cf.set("spark.submit.deployMode","client")
sc = SparkContext.getOrCreate(cf)

spark = SparkSession \
    .builder \
        .appName("null_values") \
            .config("spark.some.config.option", "some-value") \
                .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/08 14:24:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.functions import trim, col, row_number, monotonically_increasing_id
from pyspark.sql.window import Window

# Load the data stored in the file
df = spark.read.csv("dataset/2010_-_2016_School_Safety_Report_20240413.csv", header=True, inferSchema=True)

# Trim the leading and trailing whitespaces
for column in df.columns:
    df = df.withColumn(column, trim(col(column)))

# Add a unique index to each row
df_with_index = df.withColumn('INDEX', row_number().over(Window.orderBy(monotonically_increasing_id())) - 1)

df_with_index.show()

24/05/08 14:24:15 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:24:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:24:16 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:24:16 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+-------------+------+--------------------+-------------+--------------------+-------+--------------------------+--------+--------------------+---------+--------------------+-------+-----+--------+------+-----+--------+---------+------------+----------+-------------+-----------+----------+------------+--------+---------+----------+---------------+-----------------+------------+-------+----------+--------------------+-----+
|School Year|Building Code|   DBN|       Location Name|Location Code|             Address|Borough|Geographical District Code|Register|       Building Name|# Schools| Schools in Building|Major N|Oth N|NoCrim N|Prop N|Vio N|ENGroupA|   RangeA|AvgOfMajor N|AvgOfOth N|AvgOfNoCrim N|AvgOfProp N|AvgOfVio N|Borough Name|Postcode| Latitude| Longitude|Community Board|Council District |Census Tract|    BIN|       BBL|                 NTA|INDEX|
+-----------+-------------+------+--------------------+-------------+--------------------+-------+------------------------

In [3]:
from pyspark.sql.functions import when
from functools import reduce

def null_detection(df):

    # Define the list of null values
    explicit_null_list = ['0', 'N/A', '#N/A', 'n/a', 'No Data', 'Not Applicable']
    disguised_null_list = ['01/01/1900', '(999) 999-9999']

    # Replace the null values with 'Unknown'
    df_null = df
    for column in df.columns:
        df_null = df_null.withColumn(column, 
                                     when(col(column).isNull() | col(column).cast("string").isin(explicit_null_list + disguised_null_list), 
                                          'Unknown').otherwise(col(column).cast("string")))
    
    # Keep only the rows that contain at least one known value
    df_null = df_null.filter(~reduce(lambda x, y: x & y, (col(column) != 'Unknown' for column in df.columns)))
    
    return df_null

In [4]:
# Detect null values
df_null = null_detection(df_with_index)

df_null.show()

24/05/08 14:24:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:24:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:24:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:24:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/05/08 14:24:17 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+-------------+-------+--------------------+-------------+--------------------+-------+--------------------------+--------+--------------------+---------+--------------------+-------+-------+--------+-------+-------+--------+---------+------------+----------+-------------+-----------+----------+------------+--------+---------+----------+---------------+-----------------+------------+-------+----------+--------------------+-------+
|School Year|Building Code|    DBN|       Location Name|Location Code|             Address|Borough|Geographical District Code|Register|       Building Name|# Schools| Schools in Building|Major N|  Oth N|NoCrim N| Prop N|  Vio N|ENGroupA|   RangeA|AvgOfMajor N|AvgOfOth N|AvgOfNoCrim N|AvgOfProp N|AvgOfVio N|Borough Name|Postcode| Latitude| Longitude|Community Board|Council District |Census Tract|    BIN|       BBL|                 NTA|  INDEX|
+-----------+-------------+-------+--------------------+-------------+--------------------+-------+-------

24/05/08 14:24:25 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
